In [36]:
import numpy as np
import pandas as pd
from scipy import stats 
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import re

In [37]:
df = pd.read_csv("https://raw.githubusercontent.com/manipulaeHealth/desafio-machine-learning/main/dados_preco.csv")
df.head()

,descricao,criado,qtdInsumos,calculado,correto
0,30 CAP | BUPROPIONA CLORIDRATO 150MG,2020-08-05 23:19:09,1,47.5297,39.9
1,60 CAP | FINASTERIDA 1:10 1MG,2020-08-05 22:51:09,1,46.9620,43.6
2,60 CAP | NAC 250MG; SILIMARINA 150MG; SAME ...,2020-08-05 22:51:09,3,105.1390,111.4
3,60 CAP | ANASTROZOL 1:10 0.300MG,2020-08-05 22:51:09,1,49.0314,43.6
4,120 CAP | DUTASTERIDA 0.250MG; TADALAFIL 10MG,2020-08-05 22:49:09,2,161.8272,146.0


In [38]:
df.describe()

,qtdInsumos,calculado,correto
count,7121.000000,7121.000000,7121.000000
mean,3.281842,137.089552,135.891798
std,3.194301,191.266516,189.806974
min,1.000000,30.000100,33.000000
25%,1.000000,49.786100,49.500000
50%,2.000000,87.350300,83.100000
75%,4.000000,165.167000,161.200000
max,26.000000,10364.992500,10356.000000


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7121 entries, 0 to 7120
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   descricao   7121 non-null   object 
 1   criado      7121 non-null   object 
 2   qtdInsumos  7121 non-null   int64  
 3   calculado   7121 non-null   float64
 4   correto     7121 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 278.3+ KB


In [40]:
def tabelaresumo(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Nome'] = summary['index']
    summary = summary[['Nome','dtypes']]
    summary['Valores Faltantes'] = df.isnull().sum().values    
    summary['Valores Únicos'] = df.nunique().values
    summary['Primeiro Valor'] = df.loc[0].values
    summary['Segundo Valor'] = df.loc[1].values
    summary['Terceiro Valor'] = df.loc[2].values

    for name in summary['Nome'].value_counts().index:
        summary.loc[summary['Nome'] == name, 'Entropia'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 

    return summary

In [41]:
tabelaresumo(df)

Dataset Shape: (7121, 5)


,Nome,dtypes,Valores Faltantes,Valores Únicos,Primeiro Valor,Segundo Valor,Terceiro Valor,Entropia
0,descricao,object,0,7121,30 CAP | BUPROPIONA CLORIDRATO 150MG,60 CAP | FINASTERIDA 1:10 1MG,60 CAP | NAC 250MG; SILIMARINA 150MG; SAME ...,12.80
1,criado,object,0,5163,2020-08-05 23:19:09,2020-08-05 22:51:09,2020-08-05 22:51:09,12.15
2,qtdInsumos,int64,0,23,1,1,3,2.86
3,calculado,float64,0,5730,47.5297,46.962,105.139,12.31
4,correto,float64,0,2308,39.9,43.6,111.4,9.59


In [42]:
lst = []
qtdd_cap = []
for descricao in df.descricao:
    lst = descricao.split(" ")
    qtdd_cap.append(lst[0])

df["quantidade (capsulas)"] = qtdd_cap
df.head()

,descricao,criado,qtdInsumos,calculado,correto,quantidade (capsulas)
0,30 CAP | BUPROPIONA CLORIDRATO 150MG,2020-08-05 23:19:09,1,47.5297,39.9,30
1,60 CAP | FINASTERIDA 1:10 1MG,2020-08-05 22:51:09,1,46.9620,43.6,60
2,60 CAP | NAC 250MG; SILIMARINA 150MG; SAME ...,2020-08-05 22:51:09,3,105.1390,111.4,60
3,60 CAP | ANASTROZOL 1:10 0.300MG,2020-08-05 22:51:09,1,49.0314,43.6,60
4,120 CAP | DUTASTERIDA 0.250MG; TADALAFIL 10MG,2020-08-05 22:49:09,2,161.8272,146.0,120


In [43]:
lst = []
s = []
for descricao in df.descricao:
    lst = descricao.split("| ")
    s.append(lst[1])

df["formula"] = s
df.head()

,descricao,criado,qtdInsumos,calculado,correto,quantidade (capsulas),formula
0,30 CAP | BUPROPIONA CLORIDRATO 150MG,2020-08-05 23:19:09,1,47.5297,39.9,30,BUPROPIONA CLORIDRATO 150MG
1,60 CAP | FINASTERIDA 1:10 1MG,2020-08-05 22:51:09,1,46.9620,43.6,60,FINASTERIDA 1:10 1MG
2,60 CAP | NAC 250MG; SILIMARINA 150MG; SAME ...,2020-08-05 22:51:09,3,105.1390,111.4,60,NAC 250MG; SILIMARINA 150MG; SAME 50MG
3,60 CAP | ANASTROZOL 1:10 0.300MG,2020-08-05 22:51:09,1,49.0314,43.6,60,ANASTROZOL 1:10 0.300MG
4,120 CAP | DUTASTERIDA 0.250MG; TADALAFIL 10MG,2020-08-05 22:49:09,2,161.8272,146.0,120,DUTASTERIDA 0.250MG; TADALAFIL 10MG


In [44]:
lst = []
s = []
for i in df.formula:
    lst = i.split("; ")
    s.append(lst[0])

df['insumo 1'] = s
df.head()

,descricao,criado,qtdInsumos,calculado,correto,quantidade (capsulas),formula,insumo 1
0,30 CAP | BUPROPIONA CLORIDRATO 150MG,2020-08-05 23:19:09,1,47.5297,39.9,30,BUPROPIONA CLORIDRATO 150MG,BUPROPIONA CLORIDRATO 150MG
1,60 CAP | FINASTERIDA 1:10 1MG,2020-08-05 22:51:09,1,46.9620,43.6,60,FINASTERIDA 1:10 1MG,FINASTERIDA 1:10 1MG
2,60 CAP | NAC 250MG; SILIMARINA 150MG; SAME ...,2020-08-05 22:51:09,3,105.1390,111.4,60,NAC 250MG; SILIMARINA 150MG; SAME 50MG,NAC 250MG
3,60 CAP | ANASTROZOL 1:10 0.300MG,2020-08-05 22:51:09,1,49.0314,43.6,60,ANASTROZOL 1:10 0.300MG,ANASTROZOL 1:10 0.300MG
4,120 CAP | DUTASTERIDA 0.250MG; TADALAFIL 10MG,2020-08-05 22:49:09,2,161.8272,146.0,120,DUTASTERIDA 0.250MG; TADALAFIL 10MG,DUTASTERIDA 0.250MG


In [45]:
df['insumo_1']=df['insumo 1'].str.extract(r'(\D*\s?\D*)', expand = True)
df.head()

,descricao,criado,qtdInsumos,calculado,correto,quantidade (capsulas),formula,insumo 1,insumo_1
0,30 CAP | BUPROPIONA CLORIDRATO 150MG,2020-08-05 23:19:09,1,47.5297,39.9,30,BUPROPIONA CLORIDRATO 150MG,BUPROPIONA CLORIDRATO 150MG,BUPROPIONA CLORIDRATO
1,60 CAP | FINASTERIDA 1:10 1MG,2020-08-05 22:51:09,1,46.9620,43.6,60,FINASTERIDA 1:10 1MG,FINASTERIDA 1:10 1MG,FINASTERIDA
2,60 CAP | NAC 250MG; SILIMARINA 150MG; SAME ...,2020-08-05 22:51:09,3,105.1390,111.4,60,NAC 250MG; SILIMARINA 150MG; SAME 50MG,NAC 250MG,NAC
3,60 CAP | ANASTROZOL 1:10 0.300MG,2020-08-05 22:51:09,1,49.0314,43.6,60,ANASTROZOL 1:10 0.300MG,ANASTROZOL 1:10 0.300MG,ANASTROZOL
4,120 CAP | DUTASTERIDA 0.250MG; TADALAFIL 10MG,2020-08-05 22:49:09,2,161.8272,146.0,120,DUTASTERIDA 0.250MG; TADALAFIL 10MG,DUTASTERIDA 0.250MG,DUTASTERIDA


In [46]:
df['qtdd_1']=df['insumo 1'].str.extract(r'(\d.?\d?\d?\d?[G-M]?[A-Z][G-M])', expand = True)
df.head()

,descricao,criado,qtdInsumos,calculado,correto,quantidade (capsulas),formula,insumo 1,insumo_1,qtdd_1
0,30 CAP | BUPROPIONA CLORIDRATO 150MG,2020-08-05 23:19:09,1,47.5297,39.9,30,BUPROPIONA CLORIDRATO 150MG,BUPROPIONA CLORIDRATO 150MG,BUPROPIONA CLORIDRATO,150MG
1,60 CAP | FINASTERIDA 1:10 1MG,2020-08-05 22:51:09,1,46.9620,43.6,60,FINASTERIDA 1:10 1MG,FINASTERIDA 1:10 1MG,FINASTERIDA,1MG
2,60 CAP | NAC 250MG; SILIMARINA 150MG; SAME ...,2020-08-05 22:51:09,3,105.1390,111.4,60,NAC 250MG; SILIMARINA 150MG; SAME 50MG,NAC 250MG,NAC,250MG
3,60 CAP | ANASTROZOL 1:10 0.300MG,2020-08-05 22:51:09,1,49.0314,43.6,60,ANASTROZOL 1:10 0.300MG,ANASTROZOL 1:10 0.300MG,ANASTROZOL,0.300MG
4,120 CAP | DUTASTERIDA 0.250MG; TADALAFIL 10MG,2020-08-05 22:49:09,2,161.8272,146.0,120,DUTASTERIDA 0.250MG; TADALAFIL 10MG,DUTASTERIDA 0.250MG,DUTASTERIDA,0.250MG


In [47]:
df['qtdd_1_unit']=df['qtdd_1'].str.extract(r'([G-M]?[A-Z][G-M])', expand = True)
df.head()

,descricao,criado,qtdInsumos,calculado,correto,quantidade (capsulas),formula,insumo 1,insumo_1,qtdd_1,qtdd_1_unit
0,30 CAP | BUPROPIONA CLORIDRATO 150MG,2020-08-05 23:19:09,1,47.5297,39.9,30,BUPROPIONA CLORIDRATO 150MG,BUPROPIONA CLORIDRATO 150MG,BUPROPIONA CLORIDRATO,150MG,MG
1,60 CAP | FINASTERIDA 1:10 1MG,2020-08-05 22:51:09,1,46.9620,43.6,60,FINASTERIDA 1:10 1MG,FINASTERIDA 1:10 1MG,FINASTERIDA,1MG,MG
2,60 CAP | NAC 250MG; SILIMARINA 150MG; SAME ...,2020-08-05 22:51:09,3,105.1390,111.4,60,NAC 250MG; SILIMARINA 150MG; SAME 50MG,NAC 250MG,NAC,250MG,MG
3,60 CAP | ANASTROZOL 1:10 0.300MG,2020-08-05 22:51:09,1,49.0314,43.6,60,ANASTROZOL 1:10 0.300MG,ANASTROZOL 1:10 0.300MG,ANASTROZOL,0.300MG,MG
4,120 CAP | DUTASTERIDA 0.250MG; TADALAFIL 10MG,2020-08-05 22:49:09,2,161.8272,146.0,120,DUTASTERIDA 0.250MG; TADALAFIL 10MG,DUTASTERIDA 0.250MG,DUTASTERIDA,0.250MG,MG


In [50]:
df['qtdd_1_unit'].unique()

array(['MG', 'MCG', 'UI', 'MIL', nan, 'SL', 'OL', 'MK', 'LI', 'OH', 'EM',
       'AL', 'TM', 'PI', 'PL', 'MTH', 'BL', 'BI'], dtype=object)

In [ ]:
df['qtdd_1_final']=df['qtdd_1'].str.extract(r'(\d.?\d?\d?\d?)' , expand = True)
df['qtdd_1_final']=df['qtdd_1_final'].replace({'M':''}, regex=True)
df['qtdd_1_final']=df['qtdd_1_final'].replace({'F':''}, regex=True)
df['qtdd_1_final']=df['qtdd_1_final'].replace({'B':''}, regex=True)
df['qtdd_1_final']=df['qtdd_1_final'].replace({'\(':''}, regex=True)
df['qtdd_1_final']=df['qtdd_1_final'].replace({'-':''}, regex=True)
df['qtdd_1_final']=df['qtdd_1_final'].replace({' ':''}, regex=True)
df.head()

In [ ]:
df["qtdd_1_final"].unique()

In [ ]:
df["qtdd_1_final"] = df["qtdd_1_final"].astype('float')

In [ ]:
df.info()

In [ ]:
df["qtdd_1_unit"].unique()

In [ ]:
lst = []
s = []
p = []
for i in df.formula:
    lst = i.split("; ")
    s.append(lst)
    try:
        p.append(lst[1])
    except IndexError:
        p.append("NaN")   

df['insumo 2'] = p
df.head()

In [ ]:
lst = []
s = []
for i in df.formula:
    lst = i.split("; ")
    s.append(lst[2])

df['insumo 2'] = s
df.head(20)